In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/ML group Proj 2/Gopichand_Tadapaneni_Annotation.csv')

In [ ]:
dataset.head()

,Review,Label
0,We thought it was a feel good movie. My grands...,positive
1,If you like original gut wrenching laughter yo...,positive
2,I saw this movie when I was about 12 when it c...,negative
3,"I remember this film,it was the first film i h...",positive
4,This was the worst movie I saw at WorldFest an...,negative


In [ ]:
dataset['Label'] = [1 if Label == 'positive' else 0 for Label in dataset['Label'].values]
dataset.head()

,Review,Label
0,We thought it was a feel good movie. My grands...,1
1,If you like original gut wrenching laughter yo...,1
2,I saw this movie when I was about 12 when it c...,0
3,"I remember this film,it was the first film i h...",1
4,This was the worst movie I saw at WorldFest an...,0


In [ ]:
reviews = dataset['Review'].values
sentiments = dataset['Label'].values

train_reviews = reviews[:175]
train_sentiments = sentiments[:175]


test_reviews = reviews[175:]
test_sentiments = sentiments[175:]
train_reviews.shape, test_reviews.shape

((175,), (74,))

In [ ]:
print(reviews)
print(sentiments)

['We thought it was a feel good movie. My grandson said it was AMAZING!'
 'If you like original gut wrenching laughter you will like this movie. If you are young or old then you will love this movie, hell even my mom liked it.'
 "I saw this movie when I was about 12 when it came out. I recall the scariest scene was the big bird eating men dangling helplessly from parachutes right out of the air. The horror. The horror.<br /><br />As a young kid going to these cheesy B films on Saturday afternoons, I still was tired of the formula for these monster type movies that usually included the hero, a beautiful woman who might be the daughter of a professor and a happy resolution when the monster died in the end. I didn't care much for the romantic angle as a 12 year old and the predictable plots. I love them now for the unintentional humor.<br /><br />But, about a year or so later, I saw Psycho when it came out and I loved that the star, Janet Leigh, was bumped off early in the film. I sat up 

In [ ]:
# Pre-processing the text
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata


def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

def remove_accented_chars(text):                                                                    #é, è, ê
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = strip_html_tags(doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = remove_accented_chars(doc)
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()  
    norm_docs.append(doc)
  
  return norm_docs

In [ ]:
train_reviews = np.array(pre_process_corpus(train_reviews))
test_reviews = np.array(pre_process_corpus(test_reviews))

100%|██████████| 74/74 [00:00<00:00, 4749.41it/s]


In [ ]:
train_reviews

array(['we thought it was a feel good movie my grandson said it was amazing',
       'if you like original gut wrenching laughter you will like this movie if you are young or old then you will love this movie hell even my mom liked it',
       'i saw this movie when i was about 12 when it came out i recall the scariest scene was the big bird eating men dangling helplessly from parachutes right out of the air the horror the horroras a young kid going to these cheesy b films on saturday afternoons i still was tired of the formula for these monster type movies that usually included the hero a beautiful woman who might be the daughter of a professor and a happy resolution when the monster died in the end i didnt care much for the romantic angle as a 12 year old and the predictable plots i love them now for the unintentional humorbut about a year or so later i saw psycho when it came out and i loved that the star janet leigh was bumped off early in the film i sat up and took notice at that 

In [ ]:
import numpy as np
from numpy import array 
import pandas as pd
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D, Dropout
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

In [ ]:
from sklearn.model_selection import train_test_split
token = Tokenizer(num_words=5000)
token.fit_on_texts(train_reviews)
#vocab_size = len(token.word_index)+1
#token2 = Tokenizer()
#token2.fit_on_texts(test_reviews)
#vocab_size = len(token2.word_index)+1
#vocab_size


In [ ]:
token.index_word

{1: 'the',
 2: 'and',
 3: 'a',
 4: 'of',
 5: 'it',
 6: 'to',
 7: 'i',
 8: 'this',
 9: 'movie',
 10: 'is',
 11: 'was',
 12: 'in',
 13: 'that',
 14: 'for',
 15: 'but',
 16: 'with',
 17: 'you',
 18: 'film',
 19: 'as',
 20: 'so',
 21: 'on',
 22: 'not',
 23: 'its',
 24: 'one',
 25: 'all',
 26: 'like',
 27: 'have',
 28: 'just',
 29: 'about',
 30: 'from',
 31: 'be',
 32: 'at',
 33: 'good',
 34: 'some',
 35: 'his',
 36: 'are',
 37: 'see',
 38: 'if',
 39: 'my',
 40: 'an',
 41: 'very',
 42: 'bad',
 43: 'or',
 44: 'out',
 45: 'they',
 46: 'he',
 47: 'would',
 48: 'story',
 49: 'me',
 50: 'were',
 51: 'great',
 52: 'much',
 53: 'had',
 54: 'by',
 55: 'plot',
 56: 'more',
 57: 'who',
 58: 'watch',
 59: 'even',
 60: 'movies',
 61: 'up',
 62: 'what',
 63: 'time',
 64: 'there',
 65: 'no',
 66: 'how',
 67: 'we',
 68: 'love',
 69: 'seen',
 70: 'your',
 71: 'really',
 72: 'when',
 73: 'life',
 74: 'will',
 75: 'can',
 76: 'acting',
 77: 'has',
 78: 'only',
 79: 'well',
 80: 'dont',
 81: 'made',
 82: 'bec

In [ ]:
encoded_train_reviews = token.texts_to_sequences(train_reviews)
#encoded_train_reviews = np.asarray(encoded_train_reviews)
encoded_test_reviews = token.texts_to_sequences(test_reviews)
#encoded_test_reviews = np.asarray(encoded_test_reviews)
encoded_test_reviews

[[19,
  3,
  1469,
  563,
  7,
  562,
  8,
  9,
  2193,
  2205,
  64,
  36,
  60,
  13,
  27,
  81,
  49,
  14,
  249,
  1469,
  367,
  8,
  11,
  22,
  5,
  7,
  562,
  8,
  9,
  6,
  31,
  2,
  862,
  5,
  10,
  22,
  121,
  1,
  775,
  2,
  3,
  501,
  13,
  17,
  74,
  152,
  94,
  5,
  2334,
  107,
  263,
  19,
  8,
  10,
  22,
  5],
 [184, 2, 252, 9],
 [870, 344, 2461, 12, 134, 16, 3, 174, 300, 1235, 152, 4, 63],
 [8,
  9,
  11,
  132,
  42,
  76,
  2016,
  88,
  2,
  55,
  152,
  4,
  63,
  39,
  193,
  2,
  7,
  221,
  103,
  5,
  198,
  373,
  32,
  203,
  106,
  1879,
  535,
  45,
  36,
  28,
  2333,
  1,
  1053,
  22,
  1,
  18,
  2515,
  8,
  24],
 [1733, 557, 13, 188, 12, 3, 9, 12, 154, 394, 44, 416, 2, 590, 417],
 [28, 417, 1, 463, 16, 2, 111, 9, 4, 1, 177],
 [127, 2, 41],
 [1,
  90,
  12,
  1,
  9,
  513,
  44,
  6,
  31,
  1,
  90,
  1,
  1173,
  1160,
  513,
  44,
  6,
  31,
  3,
  8,
  18,
  2039,
  6,
  1,
  1160,
  12,
  816,
  128,
  22,
  152,
  70,
  63],
 [310, 

In [ ]:
# Adding 1 because of reserved 0 index
from keras_preprocessing.sequence import pad_sequences

vocab_size = len(token.word_index) + 1

maxlen = 100

X_train = pad_sequences(encoded_train_reviews , padding='post', maxlen=maxlen) #pad sequences to the maxlen:100
X_test = pad_sequences(encoded_test_reviews , padding='post', maxlen=maxlen)

X_train.shape

(175, 100)

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('/content/drive/MyDrive/ML group Proj 2/Copy of glove.6B.100d.txt')

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [ ]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in token.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
         0.82779998,  0.27061999],
       [-0.071953  ,  0.23127   ,  0.023731  , ..., -0.71894997,
         0.86894   ,  0.19539   ],
       ...,
       [ 0.031232  , -0.15669   ,  0.31434   , ...,  0.39523   ,
         0.61461002, -0.24496999],
       [ 0.59147   ,  0.09185   , -0.13879   , ...,  0.49448001,
         0.021685  ,  0.59407002],
       [-0.44532999, -0.44969001,  1.23930001, ...,  0.65855998,
         0.79881001, -0.16019   ]])

In [ ]:
from keras.layers import Dense, Conv1D , MaxPooling1D, Flatten, Dropout
def build_model():
    model = Sequential()
    embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
    model.add(embedding_layer)
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(Conv1D(64,5, activation = 'relu'))
    model.add(MaxPooling1D((2)))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
k = 10
num_val_samples = len(X_train) // 10
num_epochs = 100
all_scores = []
for i in range(k):
    print(f"Processing fold #{i}")
    val_data = X_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_sentiments[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
        [X_train[:i * num_val_samples],
         X_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_sentiments[:i * num_val_samples],
         train_sentiments[(i + 1) * num_val_samples:]],
        axis=0)
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,epochs=10, batch_size=16)
    val_mse, val_mae = model.evaluate(val_data, val_targets)
    all_scores.append(val_mae)


Processing fold #0
Epoch 1/10
10/10 [==============================] - 2s 31ms/step - loss: 0.6616 - accuracy: 0.5886
Epoch 2/10
10/10 [==============================] - 0s 29ms/step - loss: 0.5636 - accuracy: 0.6772
Epoch 3/10
10/10 [==============================] - 0s 32ms/step - loss: 0.4662 - accuracy: 0.7848
Epoch 4/10
10/10 [==============================] - 0s 32ms/step - loss: 0.3624 - accuracy: 0.8734
Epoch 5/10
10/10 [==============================] - 0s 32ms/step - loss: 0.2885 - accuracy: 0.9114
Epoch 6/10
10/10 [==============================] - 0s 32ms/step - loss: 0.2086 - accuracy: 0.9430
Epoch 7/10
10/10 [==============================] - 0s 33ms/step - loss: 0.1417 - accuracy: 0.9810
Epoch 8/10
10/10 [==============================] - 0s 35ms/step - loss: 0.1308 - accuracy: 0.9620
Epoch 9/10
10/10 [==============================] - 0s 31ms/step - loss: 0.0721 - accuracy: 0.9937
Epoch 10/10
1/1 [==============================] - 0s 259ms/step - loss: 0.8428 - accuracy

1/1 [==============================] - 0s 192ms/step - loss: 0.8162 - accuracy: 0.7647
Processing fold #5
Epoch 1/10
10/10 [==============================] - 1s 33ms/step - loss: 0.6880 - accuracy: 0.6139
Epoch 2/10
10/10 [==============================] - 0s 30ms/step - loss: 0.6040 - accuracy: 0.6709
Epoch 3/10
10/10 [==============================] - 0s 31ms/step - loss: 0.4985 - accuracy: 0.7785
Epoch 4/10
10/10 [==============================] - 0s 31ms/step - loss: 0.4369 - accuracy: 0.8291
Epoch 5/10
10/10 [==============================] - 0s 31ms/step - loss: 0.3654 - accuracy: 0.8418
Epoch 6/10
10/10 [==============================] - 0s 32ms/step - loss: 0.3038 - accuracy: 0.9114
Epoch 7/10
10/10 [==============================] - 0s 30ms/step - loss: 0.1866 - accuracy: 0.9873
Epoch 8/10
10/10 [==============================] - 0s 31ms/step - loss: 0.1282 - accuracy: 0.9810
Epoch 9/10
10/10 [==============================] - 0s 32ms/step - loss: 0.1338 - accuracy: 0.9684
Epo

1/1 [==============================] - 0s 193ms/step - loss: 0.6906 - accuracy: 0.7059
Processing fold #6
Epoch 1/10
10/10 [==============================] - 1s 33ms/step - loss: 0.6610 - accuracy: 0.6582
Epoch 2/10
10/10 [==============================] - 0s 31ms/step - loss: 0.5673 - accuracy: 0.7089
Epoch 3/10
10/10 [==============================] - 0s 32ms/step - loss: 0.4692 - accuracy: 0.7595
Epoch 4/10
10/10 [==============================] - 0s 33ms/step - loss: 0.3622 - accuracy: 0.8671
Epoch 5/10
10/10 [==============================] - 0s 31ms/step - loss: 0.2931 - accuracy: 0.9051
Epoch 6/10
10/10 [==============================] - 0s 32ms/step - loss: 0.2054 - accuracy: 0.9620
Epoch 7/10
10/10 [==============================] - 0s 32ms/step - loss: 0.1464 - accuracy: 0.9873
Epoch 8/10
10/10 [==============================] - 0s 31ms/step - loss: 0.1152 - accuracy: 0.9810
Epoch 9/10
10/10 [==============================] - 0s 32ms/step - loss: 0.0649 - accuracy: 0.9937
Epo

In [ ]:
all_scores


[0.6470588445663452,
 0.6470588445663452,
 0.7647058963775635,
 0.7647058963775635,
 0.7647058963775635,
 0.7058823704719543,
 0.7058823704719543,
 0.7647058963775635,
 0.8823529481887817,
 0.8823529481887817]

In [ ]:
np.mean(all_scores)

0.7529411911964417